## Dependencies

All the dependencies for this notebook are included in the `requirements.txt` file included in this folder.


In [1]:
import os
from pathlib import Path
import json

import geopandas as gpd
import datetime
import rasterio
import numpy as np
import pandas as pd
import elevation
import shapely

In [2]:
assets_train_df = pd.read_csv('data/assets_train.csv', index_col=0).reset_index()
assets_test_df = pd.read_csv('data/assets_test.csv', index_col=0).reset_index()

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
assets_train_df.datetime = pd.to_datetime(assets_train_df.datetime)

assets_train_df = assets_train_df.assign(
    date = lambda x: x['datetime'].dt.date,
    month = lambda x: x['datetime'].dt.month.astype('Int64'),
    dayofyear = lambda x: x['datetime'].dt.dayofyear.astype('Int64'),
)

In [4]:
assets_train_df.head(10)

,tile_id,datetime,satellite_platform,asset,file_path,geometry,date,month,dayofyear
0,2587,NaT,NaN,documentation,/home/jupyter/NF-Capstone-Crop-Classification/...,NaN,NaT,<NA>,<NA>
1,2587,NaT,NaN,field_ids,/home/jupyter/NF-Capstone-Crop-Classification/...,NaN,NaT,<NA>,<NA>
2,2587,NaT,NaN,field_info_train,/home/jupyter/NF-Capstone-Crop-Classification/...,NaN,NaT,<NA>,<NA>
3,2587,NaT,NaN,labels,/home/jupyter/NF-Capstone-Crop-Classification/...,NaN,NaT,<NA>,<NA>
4,2587,NaT,NaN,raster_values,/home/jupyter/NF-Capstone-Crop-Classification/...,NaN,NaT,<NA>,<NA>
5,2587,2017-04-01 00:00:00+00:00,s1,VH,/home/jupyter/NF-Capstone-Crop-Classification/...,"{'coordinates': [[[18.572327426397308, -33.593...",2017-04-01,4,91
6,2587,2017-04-01 00:00:00+00:00,s1,VV,/home/jupyter/NF-Capstone-Crop-Classification/...,"{'coordinates': [[[18.572327426397308, -33.593...",2017-04-01,4,91
7,2587,2017-04-06 00:00:00+00:00,s1,VH,/home/jupyter/NF-Capstone-Crop-Classification/...,"{'coordinates': [[[18.572327426397308, -33.593...",2017-04-06,4,96
8,2587,2017-04-06 00:00:00+00:00,s1,VV,/home/jupyter/NF-Capstone-Crop-Classification/...,"{'coordinates': [[[18.572327426397308, -33.593...",2017-04-06,4,96
9,2587,2017-04-13 00:00:00+00:00,s1,VH,/home/jupyter/NF-Capstone-Crop-Classification/...,"{'coordinates': [[[18.572327426397308, -33.593...",2017-04-13,4,103


In [5]:
tile_ids_train = assets_train_df['tile_id'].unique()

In [6]:
fields_train = gpd.read_file('data/data_fields/fields_train.geojson')
fields_train.head()

,field_id,label,tile_id,elevation,field_area_km2,geometry
0,3020,2,2587,197.333333,0.001702,"POLYGON ((18.57295 -33.57073, 18.57293 -33.571..."
1,99466,2,2587,173.574074,0.042403,"POLYGON ((18.57517 -33.57230, 18.57513 -33.573..."
2,15902,5,2587,149.222222,0.029503,"POLYGON ((18.58087 -33.57259, 18.58087 -33.572..."
3,38846,3,2587,89.761364,0.069601,"POLYGON ((18.59842 -33.57285, 18.59842 -33.572..."
4,87981,1,2587,90.272727,0.022603,"POLYGON ((18.59189 -33.58714, 18.59186 -33.587..."


In [7]:
tiles_train = gpd.read_file('data/tiles_train.geojson')
tiles_train.head()

,tile_id,all_days,sunny_days,sun_rate,tiles_closest,tile_label_dist,close_label_dist,geometry
0,2587,"{ 91, 101, 111, 121, 131, 141, 151, 161, 171, ...","{101, 111, 121, 141, 171, 186, 191, 211, 221, ...",0.552632,"{1762, 2637, 1624, 1690, 955, 1628}","{'1': 0.5833333333333334, '2': 0.25, '3': 0.08...","{'1': 0.15300127713920816, '2': 0.433001194208...","POLYGON ((18.57233 -33.59307, 18.59989 -33.593..."
1,1302,"{ 91, 101, 111, 121, 131, 141, 151, 161, 171, ...","{101, 111, 131, 141, 151, 171, 181, 186, 191, ...",0.605263,"{195, 771, 1739, 2574, 1007, 272, 1271}","{'1': 0, '2': 0, '3': 0.42857142857142855, '4'...","{'1': 0, '2': 0, '3': 0.21252587991718427, '4'...","POLYGON ((19.10229 -32.03239, 19.12939 -32.032..."
2,1130,"{ 91, 101, 111, 121, 131, 141, 151, 161, 171, ...","{101, 111, 121, 141, 151, 171, 186, 191, 211, ...",0.578947,"{1511, 1287, 2121, 1899, 1261, 920, 2074, 799}","{'1': 0.21875, '2': 0.03125, '3': 0, '4': 0, '...","{'1': 0.29269551396724713, '2': 0.065929147653...","POLYGON ((18.66554 -33.20246, 18.69299 -33.202..."
3,486,"{ 91, 94, 101, 104, 111, 114, 121, 124, 131, ...","{91, 101, 104, 111, 114, 121, 124, 134, 141, 1...",0.684211,"{2127, 1776, 239, 1846, 2327, 2265, 1021, 253}","{'1': 0, '2': 0.05555555555555555, '3': 0.2407...","{'1': 0.10524015709445268, '2': 0.068466648931...","POLYGON ((18.73576 -32.60354, 18.76302 -32.604..."
4,1554,"{ 91, 94, 101, 104, 111, 114, 121, 124, 131, ...","{101, 104, 111, 114, 121, 124, 134, 141, 144, ...",0.644737,"{1184, 1927, 2440, 1457, 1267, 603, 1565}","{'1': 0, '2': 0, '3': 0, '4': 0, '5': 0.3125, ...","{'1': 0.12448979591836733, '2': 0.025801045055...","POLYGON ((18.66234 -32.27899, 18.68950 -32.279..."


In [8]:
assets_train_stacked_df = pd.read_csv('data/assets_stacked_8days_train.csv')
assets_train_stacked_df.head()

,tile_id,datetime,satellite_platform,asset,file_path,date,month,dayofyear
0,2587,2017-04-11 00:00:00+00:00,s2,s2_stacked,/home/jupyter/NF-Capstone-Crop-Classification/...,2017-04-11,4,101
1,2587,2017-05-01 00:00:00+00:00,s2,s2_stacked,/home/jupyter/NF-Capstone-Crop-Classification/...,2017-05-01,5,121
2,2587,2017-07-05 00:00:00+00:00,s2,s2_stacked,/home/jupyter/NF-Capstone-Crop-Classification/...,2017-07-05,7,186
3,2587,2017-08-09 00:00:00+00:00,s2,s2_stacked,/home/jupyter/NF-Capstone-Crop-Classification/...,2017-08-09,8,221
4,2587,2017-09-18 00:00:00+00:00,s2,s2_stacked,/home/jupyter/NF-Capstone-Crop-Classification/...,2017-09-18,9,261


In [9]:
channel_to_number = {"B01":0, "B02":1, "B03":2, "B04":3, "B05":4, "B06":5, "B07":6, "B08":7,
                     "B09":8, "B11":9, "B12":10, "B8A":11, "CLM":None, "VEG_IDX": 12, "MOIST_IDX": 13, "NDVI_IDX": 14}
number_to_channel = {number: channel for channel, number in channel_to_number.items()}

In [13]:
def mean_var(image, number_to_channel):
    """
    extracts mean and variance for each layer of 'image'
    
    Arguments: - 'src': np.array in the form of (nr_channels, height_image, width_image)
    """
    
    # count nr. of channels in 'src'
    nr_chan = image.shape[0]
    
    mean = {number_to_channel[chan] + "_MEAN": 0 for chan in range(0, nr_chan)}
    var = {number_to_channel[chan] + "_VAR": 0 for chan in range(0, nr_chan)}
    
    for chan in range(0, nr_chan):
        data = image[chan, image[chan, :, :] != 0]
        if(len(data.flatten()) > 0):
            mean[number_to_channel[chan] + "_MEAN"] = np.mean(data.flatten())
            var[number_to_channel[chan] + "_VAR"] = np.var(data.flatten())
    
    return (mean, var)

In [10]:
def create_vegindex(image, channel_to_number):
    image_B8 = image[channel_to_number['B08'],:,:]
    image_B4 = image[channel_to_number['B04'],:,:]
    non_zero = (image_B8 + image_B4 != 0)
    vegetation_index = np.zeros((image.shape[1], image.shape[2]))
    vegetation_index[non_zero] = np.nan_to_num((image_B8[non_zero] - image_B4[non_zero]) / (image_B8[non_zero] + image_B4[non_zero]))
    return vegetation_index

In [11]:
def create_moistureindex(image, channel_to_number):
    image_B8A = image[channel_to_number['B8A'],:,:]
    image_B11 = image[channel_to_number['B11'],:,:]
    non_zero = (image_B8A + image_B11 != 0)
    moisture_index = np.zeros((image.shape[1], image.shape[2]))
    moisture_index[non_zero] = np.nan_to_num((image_B8A[non_zero] - image_B11[non_zero]) / (image_B8A[non_zero] + image_B11[non_zero]))
    return moisture_index

In [12]:
def create_ndviindex(image, channel_to_number):
    image_B08 = image[channel_to_number['B08'],:,:]
    image_B04 = image[channel_to_number['B04'],:,:]
    non_zero = (image_B08 + image_B04 != 0)
    ndvi_index = np.zeros((image.shape[1], image.shape[2]))
    ndvi_index[non_zero] = np.nan_to_num((image_B08[non_zero] - image_B04[non_zero]) / (image_B08[non_zero] + image_B04[non_zero]))
    return ndvi_index

In [23]:
import rasterio.features
from rasterio.plot import show
from rasterio.mask import mask

def mask_mean_var(geom, src, geom_crs, channel_to_number, number_to_channel):
    """
    extracts mean and variance for each layer of 'path' after masking it with the geometry of 'geom'
    
    Arguments: - 'src': rasterio DataSetReader
               - 'geom': shapely geometry (Polygon) that lies on the 'src' AND USES THE SAME CRS!
    """
    
    
    geom = rasterio.warp.transform_geom(geom_crs, src.crs, geom, precision=6)
    image_masked, _ = mask(src, shapes=[geom], crop=True)

    # remove cloud coverage channel
    image_masked = np.delete(image_masked,12, axis=0)
    
    image_masked_with_idx = np.zeros((image_masked.shape[0] + 2, image_masked.shape[1], image_masked.shape[2]))
    image_masked_with_idx[0:image_masked.shape[0],:,:] = image_masked
    image_masked_with_idx[channel_to_number['VEG_IDX'],:,:] = create_vegindex(image_masked, channel_to_number)
    image_masked_with_idx[channel_to_number['MOIST_IDX'],:,:] = create_moistureindex(image_masked, channel_to_number)
    
    mean, var = mean_var(image_masked_with_idx, number_to_channel)
    return mean, var

In [30]:
#fields_train_mean_var1 = pd.read_csv('fields_train_mean_var_data1.csv')
#fields_train_mean_var2 = pd.read_csv('fields_train_mean_var_data2.csv')
#fields_train_mean_var3 = pd.read_csv('fields_train_mean_var_data3.csv')

In [31]:
#fields_train_mean_var = pd.concat([fields_train_mean_var, fields_train_mean_var2, fields_train_mean_var2], axis=0)

In [33]:
#fields_train_mean_var = fields_train_mean_var.drop('neighboor_label_sun_rate', axis=1)

In [19]:
#tile_ids_train2 = np.setdiff1d(tile_ids_train, fields_train_mean_var.tile_id.unique())

839

In [24]:
def mask_mean_var_wrapper(row, src, geom_crs, channel_to_number, number_to_channel, day):
    mean, var = mask_mean_var(row.geometry, src, geom_crs, channel_to_number, number_to_channel)
    mean_var_day = {key + "_" + str(day): value for key, value in {**mean, **var}.items()}
    return mean_var_day

def get_sun_labeldist(tile_id, nr_fields, tiles_train):
    tile = tiles_train.query('tile_id == ' + str(tile_id)).iloc[0]
    dict_temp = {'neighboor_label_' + key: value for key, value in tile.close_label_dist.items()}
    dict_temp['sun_rate'] = tile.sun_rate
    
    df_temp = pd.DataFrame(dict_temp, index=[0])
    
    return pd.DataFrame(np.repeat(df_temp.values, nr_fields, axis=0), columns=df_temp.columns)

fields_train_mean_var = pd.DataFrame()

for tile_id in tile_ids_train[:5]:
    clear_output(wait=True)
    print(tile_id)
    
    tile_df = assets_train_stacked_df.query('tile_id == @tile_id')
    tile_days = list(tile_df['dayofyear'])
    
    tile_fields_df = fields_train.query('tile_id == @tile_id')
    
    right_df = get_sun_labeldist(tile_id, tile_fields_df.shape[0], tiles_train)
    right_df.index = tile_fields_df.index
    
    tile_fields_df = pd.concat([tile_fields_df, right_df], axis=1) 
    
    for day_nr, day  in enumerate(tile_days):
        path = tile_df.query('dayofyear == @day').iloc[0,4]
        src = rasterio.open(path)
        geom_crs = 'EPSG:4326'
        
        
        right_df = tile_fields_df.apply(mask_mean_var_wrapper, args=(src, geom_crs, channel_to_number, number_to_channel, day_nr,), axis='columns', result_type='expand')
        tile_fields_df = pd.concat([tile_fields_df, right_df], axis=1)   
    
        
    fields_train_mean_var = pd.concat([fields_train_mean_var, tile_fields_df], axis = 0)

fields_train_mean_var.to_csv('fields_train_mean_var_data_old.csv', index=False)

1554


In [14]:
import rasterio.features
from rasterio.plot import show
from rasterio.mask import mask

def mask_mean_var2(image_stacked, field_ids_tile, label_field, channel_to_number, number_to_channel):
    """
    extracts mean and variance for each layer of 'path' after masking it with the geometry of 'geom'
    
    Arguments: - 'src': rasterio DataSetReader
               - 'geom': shapely geometry (Polygon) that lies on the 'src' AND USES THE SAME CRS!
    """
    
    # remove cloud coverage channel
    image_stacked = np.delete(image_stacked, 12, axis=0)
      
    image_with_idx = np.zeros((image_stacked.shape[0] + 3, image_stacked.shape[1], image_stacked.shape[2]))
    image_with_idx[0:image_stacked.shape[0],:,:] = image_stacked
    
    image_with_idx[channel_to_number['VEG_IDX'],:,:] = create_vegindex(image_stacked, channel_to_number)
    image_with_idx[channel_to_number['MOIST_IDX'],:,:] = create_moistureindex(image_stacked, channel_to_number)
    image_with_idx[channel_to_number['NDVI_IDX'],:,:] = create_ndviindex(image_stacked, channel_to_number)
    
    
    mask = (field_ids_tile != label_field)
    mask_broad = np.broadcast_to(mask, image_with_idx.shape)
    image_masked = ma.masked_array(image_with_idx, mask=mask_broad)
    
    mean, var = mean_var(image_masked, number_to_channel)
    return mean, var    

In [21]:
import rasterio.features
from rasterio.plot import show
from rasterio.mask import mask

def mask_mean_var3(image_stacked, field_ids_tile, field_id, channel_to_number, number_to_channel):
    """
    extracts mean and variance for each layer of 'path' after masking it with the geometry of 'geom'
    

    """
    image_stacked = np.delete(image_stacked, 12, axis=0)
    
    mask = (field_ids_tile != field_id)
    
    top = np.min(np.where(np.sum(mask, axis=1) != mask.shape[1]))
    bottom = np.max(np.where(np.sum(mask, axis=1) != mask.shape[1]))
    left = np.min(np.where(np.sum(mask, axis=0) != mask.shape[0]))
    right = np.max(np.where(np.sum(mask, axis=0) != mask.shape[0]))
    
    mask_broad = np.broadcast_to(mask, image_stacked.shape)
    
    image_masked = ma.masked_array(image_stacked, mask=mask_broad).filled(fill_value=0)
    image_cropped = image_masked[:,top:(bottom+1),left:(right+1)]
    
    # remove cloud coverage channel    
      
    image_with_idx = np.zeros((image_cropped.shape[0] + 3, image_cropped.shape[1], image_cropped.shape[2]))
    image_with_idx[0:image_cropped.shape[0],:,:] = image_cropped
    
    image_with_idx[channel_to_number['VEG_IDX'],:,:] = create_vegindex(image_cropped, channel_to_number)
    image_with_idx[channel_to_number['MOIST_IDX'],:,:] = create_moistureindex(image_cropped, channel_to_number)
    image_with_idx[channel_to_number['NDVI_IDX'],:,:] = create_ndviindex(image_cropped, channel_to_number)
    
    mean, var = mean_var(image_with_idx, number_to_channel)
    return mean, var    

In [26]:
"""
crop the images with a different technique, get the mask from the label.tif file directly
"""
import numpy.ma as ma

def mask_mean_var_wrapper2(row, image_stacked, field_ids_tile, channel_to_number, number_to_channel, day):
    mean, var = mask_mean_var3(image_stacked, field_ids_tile[0,:,:], row.field_id, channel_to_number, number_to_channel)
    mean_var_day = {key + "_" + str(day): value for key, value in {**mean, **var}.items()}
    return mean_var_day

def get_sun_labeldist(tile_id, nr_fields, tiles_train):
    tile = tiles_train.query('tile_id == ' + str(tile_id)).iloc[0]
    dict_temp = {'neighboor_label_' + key: value for key, value in tile.close_label_dist.items()}
    dict_temp['sun_rate'] = tile.sun_rate
    
    df_temp = pd.DataFrame(dict_temp, index=[0])
    
    return pd.DataFrame(np.repeat(df_temp.values, nr_fields, axis=0), columns=df_temp.columns)

fields_train_mean_var = pd.DataFrame()

i = 0
for tile_id in tile_ids_train:
    i += 1
    clear_output(wait=True)
    print(f'Tile Nr. {i} of {len(tile_ids_train)}')
    
    tile_df = assets_train_stacked_df.query('tile_id == @tile_id')
    tile_days = list(tile_df['dayofyear'])
    
    tile_fields_df = fields_train.query('tile_id == @tile_id')
    
    right_df = get_sun_labeldist(tile_id, tile_fields_df.shape[0], tiles_train)
    right_df.index = tile_fields_df.index
    
    tile_fields_df = pd.concat([tile_fields_df, right_df], axis=1) 
    
    field_ids_tile = rasterio.open(assets_train_df.query('tile_id == @tile_id & asset == "field_ids"').iloc[0,4]).read()
    
    for day_nr, day  in enumerate(tile_days):
        path = tile_df.query('dayofyear == @day').iloc[0,4]
        image_stacked = rasterio.open(path).read()
        
        
        right_df = tile_fields_df.apply(mask_mean_var_wrapper2, args=(image_stacked, field_ids_tile, channel_to_number, number_to_channel, day_nr,), axis='columns', result_type='expand')
        tile_fields_df = pd.concat([tile_fields_df, right_df], axis=1)   
    
        
    fields_train_mean_var = pd.concat([fields_train_mean_var, tile_fields_df], axis = 0)
    
    
fields_train_mean_var.to_csv('fields_train_mean_var_data_new.csv', index=False)

Tile Nr. 2650 of 2650
